# Delta Lake

### Objectives
1. Create a Delta Table
2. Understand the transaction Log
3. Read data from your Delta Table
4. Update data in your Delta Table
5. Access previous versions of table using time travel
6. Vacuum

### Resources
- [Delta Table](https://docs.delta.io/latest/quick-start.html#create-a-table)  
- [Transaction Log](https://databricks.com/blog/2019/08/21/diving-into-delta-lake-unpacking-the-transaction-log.html)  
- [Time Travel](https://databricks.com/blog/2019/02/04/introducing-delta-time-travel-for-large-scale-data-lakes.html)


In [ ]:
import pandas as pd

data = [{
    "event_timestamp": 1650000000000,
    "user_id": "u1",
    "device": "Android",
    "geo.state": "CA",
    "ecommerce.purchase_revenue_in_usd": 123.45,
    "ecommerce.total_item_quantity": 2,
    "traffic_source": "ads"
}]

df = pd.DataFrame(data)
csv_path = "/dbfs/tmp/events.csv"
df.to_csv(csv_path, index=False)

events_df = (spark.read
             .option("header", True)
             .option("inferSchema", True)
             .csv(csv_path)
             .withColumnRenamed("geo.state", "state")
             .withColumnRenamed("ecommerce.purchase_revenue_in_usd", "revenue")
             .withColumnRenamed("ecommerce.total_item_quantity", "quantity"))

events_df.cache()
display(events_df)

In [ ]:
delta_path = "/tmp/delta/events"
events_df.write.format("delta").mode("overwrite").save(delta_path)

In [ ]:
events_df.write.format("delta").mode("overwrite").saveAsTable("delta_events")

In [ ]:
from pyspark.sql.functions import col

state_events_df = events_df.withColumn("state", col("state"))
state_events_df.write.format("delta").mode("overwrite").partitionBy("state").option("overwriteSchema", "true").save(delta_path)

In [ ]:
display(dbutils.fs.ls(delta_path))

In [ ]:
display(dbutils.fs.ls(f"{delta_path}/_delta_log/"))

In [ ]:
display(spark.read.json(f"{delta_path}/_delta_log/00000000000000000000.json"))

In [ ]:
display(spark.read.json(f"{delta_path}/_delta_log/00000000000000000001.json"))

In [ ]:
display(dbutils.fs.ls(f"{delta_path}/state=CA/"))

In [ ]:
df = spark.read.format("delta").load(delta_path)
display(df)

In [ ]:
df_update = state_events_df.filter(col("device").isin(["Android", "iOS"]))
df_update.write.format("delta").mode("overwrite").save(delta_path)

In [ ]:
df = spark.read.format("delta").load(delta_path)
display(df)

In [ ]:
display(dbutils.fs.ls(f"{delta_path}/state=CA/"))

In [ ]:
from delta.tables import *
spark.sql("DROP TABLE IF EXISTS train_delta")
spark.sql(f"CREATE TABLE train_delta USING DELTA LOCATION '{delta_path}'")

In [ ]:
# MAGIC %sql
DESCRIBE HISTORY train_delta

In [ ]:
df = spark.read.format("delta").option("versionAsOf", 0).load(delta_path)
display(df)

In [ ]:
spark.conf.set("spark.databricks.delta.retentionDurationCheck.enabled", "false")
delta_table = DeltaTable.forPath(spark, delta_path)
delta_table.vacuum(0)

In [ ]:
display(dbutils.fs.ls(delta_path + "/state=CA/"))